<a href="https://colab.research.google.com/github/MrKozelberg/wavefunction_perceptron/blob/laplacian/laplacian.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calculation of the Laplace operator for a multilayer perceptron

In [1]:
# imports
import numpy as np  # to work with arrays
import matplotlib.pyplot as plt  # to make figures

# PyTorch
import torch
from torch import nn


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [3]:
def f0(x):
    return torch.tanh(x)
  
def f1(x):
  return 1/torch.cosh(x)**2
  
def f2(x):
  return -2*torch.sinh(x)/torch.cosh(x)**3

In [35]:
class WaveFunction(nn.Module):

  def __init__(self, N, layersizes):
    self.N = N  # size of the input vector
    self.layersizes = layersizes  # sizes of hidden layers
    self.l = len(self.layersizes)  # number of hidden layers

    super(WaveFunction, self).__init__()
        
    self.linear_tanh_stack = [None] * (self.l + 1) * 2
    for i in range(self.l + 1):
      if i == 0:
        self.linear_tanh_stack[2*i] = nn.Linear(N, self.layersizes[i])
      elif i == self.l:
        self.linear_tanh_stack[2*i] = nn.Linear(self.layersizes[i-1], 1)
      else:
        self.linear_tanh_stack[2*i] = nn.Linear(self.layersizes[i-1],
                                                self.layersizes[i])
      self.linear_tanh_stack[2*i+1] = nn.Tanh()
    
    self.h = [None] * (self.l + 1)  # outputs of the layers

  
  def forward(self, x):
    for i in range(self.l + 1):
      if i == 0:
        self.h[i] = self.linear_tanh_stack[2*i+1](
            self.linear_tanh_stack[2*i](x))
      else:
        self.h[i] = self.linear_tanh_stack[2*i+1](
            self.linear_tanh_stack[2*i](self.h[i-1]))
      
    return self.h[-1]


  def grad(self, x):
    """
    finds the gradient of the wave function at a given point x
    """
    _ = self.forward(x)  # computes outputs of the layers #
    gradient = torch.zeros(x.shape) #[None] * self.N #
    for t in range(self.N):
      dh_dx = torch.zeros(self.N) #
      dh_dx[t] = 1 # 

      for i in range(self.l + 1):
        w = self.linear_tanh_stack[2*i].weight
        b = self.linear_tanh_stack[2*i].bias

        if i == 0:
          dh_dx = f1(x @ w.T + b) * (dh_dx @ w.T)
        else:
          dh_dx = f1(self.h[i-1] @ w.T + b) * (dh_dx @ w.T)   
        
      # print(dh_dx.shape, gradient[t].shape)
      
      gradient[:,t] = dh_dx.reshape(-1)

    return gradient
  

  def laplac(self, x):
    """
    finds the Laplacian of the wave function at a given point x
    """
    y = self.forward(x)  # computes outputs of the layers
    laplacian = 0.0
    for t in range(self.N):
      dh_dx = torch.zeros(self.N)
      dh_dx[t] = 1

      d2h_dx2 = torch.zeros(self.N)

      for i in range(self.l + 1):
        w = self.linear_tanh_stack[2*i].weight
        b = self.linear_tanh_stack[2*i].bias

        # firstly, the new value of the second derivative computes as it is 
        # used the previous value of the first derivative

        if i == 0:
          d2h_dx2 = f2(x @ w.T + b) * (dh_dx @ w.T)**2
        else:
          d2h_dx2 = f2(self.h[i-1] @ w.T + b) * (dh_dx @ w.T)**2\
                    + f1(self.h[i-1] @ w.T + b) * (d2h_dx2 @ w.T)

        if i == 0:
          dh_dx = f1(x @ w.T + b) * (dh_dx @ w.T)
        else:
          dh_dx = f1(self.h[i-1] @ w.T + b) * (dh_dx @ w.T) 
      
      # print(d2h_dx2)
      laplacian += d2h_dx2
    
    return laplacian


## Test

In [36]:
wf = WaveFunction(5, [5]).to(device)
# wf.linear_tanh_stack

### Definition of the weights and biases

In [37]:
with torch.no_grad():
  wf.linear_tanh_stack[0].weight[:] =torch.nn.parameter.Parameter(torch.diag(torch.ones(wf.linear_tanh_stack[0].weight.shape[0])))
  wf.linear_tanh_stack[0].bias[:] =torch.nn.parameter.Parameter(torch.zeros(wf.linear_tanh_stack[0].bias.shape))
  wf.linear_tanh_stack[2].weight[:] =torch.nn.parameter.Parameter(torch.diag(torch.ones(wf.linear_tanh_stack[2].weight.shape[0])))
  wf.linear_tanh_stack[2].bias[:] =torch.nn.parameter.Parameter(torch.zeros(wf.linear_tanh_stack[2].bias.shape))

### Test the neural network

In [38]:
x = torch.rand((3,5))

y_an = [torch.tanh(torch.sum(torch.tanh(x[i]))) for i in range(len(x))]

y_an, wf(x)

([tensor(0.9898), tensor(0.8906), tensor(0.9795)], tensor([[0.9898],
         [0.8906],
         [0.9795]], grad_fn=<TanhBackward0>))

### Test of its gradient

In [40]:
yx_an = [1/torch.cosh(torch.sum(torch.tanh(x[i])))**2 * 1/torch.cosh(x[i])**2 for i in range(len(x))]

yx_an, wf.grad(x)

([tensor([0.0134, 0.0088, 0.0135, 0.0122, 0.0202]),
  tensor([0.2010, 0.2002, 0.2013, 0.1965, 0.1082]),
  tensor([0.0364, 0.0177, 0.0387, 0.0379, 0.0175])],
 tensor([[0.0134, 0.0088, 0.0135, 0.0122, 0.0202],
         [0.2010, 0.2002, 0.2013, 0.1965, 0.1082],
         [0.0364, 0.0177, 0.0387, 0.0379, 0.0175]], grad_fn=<CopySlices>))

### Finaly, we test its laplacian

In [41]:
yxx_an = [torch.sum(-2*yx_an[i]*(torch.tanh(x[i]) + y_an[i]/torch.cosh(x[i])**2)) for i in range(len(x))]

yxx_an, wf.laplac(x)

([tensor(-0.1604), tensor(-1.9178), tensor(-0.3474)], tensor([[-0.1604],
         [-1.9178],
         [-0.3474]], grad_fn=<AddBackward0>))